In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import keras 
from keras import layers

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
data = pd.read_csv("../input/PRSA_data_2010.1.1-2014.12.31.csv")

In [ ]:
data.info()

In [ ]:
data[data['pm2.5'].isna()]#找出数值为空的pm2.5

In [ ]:
data = data.iloc[24:].copy()#去掉最前面的24个数据

In [ ]:
data.fillna(method = 'ffill',inplace = True)  #将空白的地方进行合理的填充   此处填充为前一天的pm2.5

In [ ]:
data.drop('No', axis=1, inplace=True) #将数据的第一列编号删除   （已删除 此处不再不重复运行）

In [ ]:

import datetime

In [ ]:
data['time'] = data.apply(lambda x: datetime.datetime(year=x['year'],
                                       month=x['month'],
                                       day=x['day'], 
                                       hour=x['hour']), 
                                        axis=1)   #将数据中的年月日进行合并

In [ ]:
data.set_index('time', inplace=True)   #将合并后的数据设置为索引   inplace=true  既是表示更替

In [ ]:
data

In [ ]:
data.drop(columns=['year', 'month', 'day', 'hour'], inplace=True)

In [ ]:
data.head()

In [ ]:
data.cbwd.unique()

In [ ]:
data = data.join(pd.get_dummies(data.cbwd))   #对cbwd进行独热编码  

In [ ]:
data

In [ ]:
del data['cbwd']

In [ ]:
data['pm2.5'][-1000:].plot()

In [ ]:
data['TEMP'][-1000:].plot()

In [ ]:
data.head(3)

In [ ]:
sequence_length = 5*24
delay = 24

In [ ]:
data_ = []
for i in range(len(data) - sequence_length - delay):
    data_.append(data.iloc[i: i + sequence_length + delay])  #迭代取数据

In [ ]:
data_ = np.array([df.values for df in data_])     #将数据上行迭代取出的数据处理成np.array的格式

In [ ]:
data_

In [ ]:
np.random.shuffle(data_)

In [ ]:
x = data_[:, :-delay, :]   #每一组数据用前面5天的预测第六天的
y = data_[:, -1, 0]

In [ ]:
split_boundary = int(data_.shape[0] * 0.8)   #百分之80的数据用来训练

In [ ]:
train_x = x[: split_boundary]
test_x = x[split_boundary:]

train_y = y[: split_boundary]
test_y = y[split_boundary:]

In [ ]:
train_x.shape, test_x.shape, train_y.shape, test_y.shape

In [ ]:
mean = train_x.mean(axis=0)
std = train_x.std(axis=0)
train_x = (train_x - mean)/std
test_x = (test_x - mean)/std                       #将数据进行归一化处理

In [ ]:
batch_size = 128

In [ ]:
train_x.shape[1:]

In [ ]:
model = keras.Sequential()
model.add(layers.Flatten(input_shape=(train_x.shape[1:])))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(1))

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(), loss='mae')
history = model.fit(train_x, train_y,
                    batch_size = 128,
                    epochs=50,
                    validation_data=(test_x, test_y)
                    )

In [ ]:
plt.plot(history.epoch, history.history.get('loss'), 'y', label='Training loss')
plt.plot(history.epoch, history.history.get('val_loss'), 'b', label='Test loss')
plt.legend()   #由图可见存在过拟合

In [ ]:
model = keras.Sequential()
model.add(layers.LSTM(32, input_shape=(train_x.shape[1:])))
model.add(layers.Dense(1))

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(), loss='mae')

In [ ]:
history = model.fit(train_x, train_y,
                    batch_size = 128,
                    epochs=200,
                    validation_data=(test_x, test_y))

# 用rnn lstm试一下


In [ ]:
model = keras.Sequential()
model.add(layers.LSTM(32, input_shape=(train_x.shape[1:]), return_sequences=True))
model.add(layers.LSTM(32, return_sequences=True))
model.add(layers.LSTM(32))
model.add(layers.Dense(1))
model.compile(optimizer=keras.optimizers.Adam(), loss='mae')
learning_rate_reduction = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.5, min_lr=0.00001)
history = model.fit(train_x, train_y,
                    batch_size = 128,
                    epochs=200,
                    validation_data=(test_x, test_y),
                    callbacks=[learning_rate_reduction])

In [ ]:
layers.LSTM(units, activation='tanh',                         #LSTM是内置的有默认激活函数tanh  我们平常一般用于优化的在层内的dropout
                                                            #和层间的layers.Dropout     #层内的对数据防止过拟合的用recurrent_dropout,
            recurrent_activation='hard_sigmoid', 
            use_bias=True, 
            kernel_initializer='glorot_uniform', 
            recurrent_initializer='orthogonal',
            bias_initializer='zeros', 
            unit_forget_bias=True, 
            kernel_regularizer=None, 
            recurrent_regularizer=None, 
            bias_regularizer=None, 
            activity_regularizer=None, 
            kernel_constraint=None,
            recurrent_constraint=None, 
            bias_constraint=None, 
            dropout=0.0, 
            recurrent_dropout=0.0, 
            implementation=1, 
            return_sequences=False,
            return_state=False, 
            go_backwards=False, 
            stateful=False, 
            unroll=False, 
            **kwargs)